In [1]:

from copy import deepcopy

import torch.optim as optim
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
import time

In [2]:
from senmodel.model.utils import *
from senmodel.metrics.nonlinearity_metrics import *
from senmodel.metrics.edge_finder import *

In [3]:
torch.manual_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
def get_params_amount(model):
    amount = 0
    for linear in model.embed_linears:
        amount += linear.weight_size[0] * linear.weight_size[0]
    amount += model.weight_size[0] * model.weight_size[1]
    return amount

In [5]:
def get_zero_params_amount(model, eps=1e-8):
    amount = 0
    for linear in model.embed_linears:
        amount += linear.weight_values[linear.weight_values < eps].shape[0]
    amount += model.weight_values[model.weight_values < eps].shape[0]
    return amount

In [6]:
def train_sparse_recursive(model, train_loader, val_loader, num_epochs, metric, edge_replacement_func=None, window_size=5, threshold=0.02):
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    replace_epoch = [0]
    val_losses = []
    len_choose = get_model_last_layer(model).count_replaces

    for epoch in range(num_epochs):
        t0 = time.time()
        model.train()
        train_loss = 0
        for i, (inputs, targets) in enumerate(tqdm(train_loader)):
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            optimizer.zero_grad()
            loss.backward()

            # if len(len_choose) > 3 and i > window_size:
            #     freeze_all_but_last(model)

            optimizer.step()
            train_loss += loss.item()
        
        if len(replace_epoch) > 1:
            for g in optimizer.param_groups:
                g['lr'] *= 0.9

        train_loss /= len(train_loader)
        train_time = time.time() - t0

        model.eval()
        val_loss = 0
        all_targets = []
        all_preds = []
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

                preds = torch.argmax(outputs, dim=1)
                all_targets.extend(targets.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())

        val_loss /= len(val_loader)
        val_accuracy = accuracy_score(all_targets, all_preds)

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
        new_l = dict()
        # if edge_replacement_func and epoch % 8 == 0 and epoch != 0:
        #     new_l = edge_replacement_func(model, optimizer, val_loader, metric)
        val_losses.append(val_loss)
        if len(val_losses) > window_size and epoch - replace_epoch[-1] > 8:
            recent_changes = [abs(val_losses[i] - val_losses[i - 1]) for i in range(-window_size, 0)]
            avg_change = sum(recent_changes) / window_size
            if avg_change < threshold:
                len_ch = len_choose[-1]
                new_l = edge_replacement_func(model, optimizer, val_loader, metric, 0.3, 'mean', len_ch)
                len_choose = get_model_last_layer(model).count_replaces
                replace_epoch += [epoch]
                if len(replace_epoch) == 2:
                    for g in optimizer.param_groups:
                        g['lr'] *= 200
            print(torch.unique(get_model_last_layer(model).weight_indices[0]))

        print(new_l)
        params_amount = get_params_amount(model.fc1)
        zero_params_amount = get_zero_params_amount(model.fc1)
        wandb.log({'val loss': val_loss, 'val accuracy': val_accuracy,
                    'train loss': train_loss, 'params amount': params_amount,
                      'zero params amount': zero_params_amount, 'train time': train_time,
                        'params ratio': (params_amount - zero_params_amount) / params_amount,
                          'lr': optimizer.param_groups[0]['lr']} | new_l)


def edge_replacement_func_new_layer(model, optim, val_loader, metric, choose_threshold, aggregation_mode='mean', len_choose=None):
    layer = get_model_last_layer(model)
    ef = EdgeFinder(metric, val_loader, device, aggregation_mode)
    vals = ef.calculate_edge_metric_for_dataloader(model, len_choose, False)
    print("Edge metrics:", vals, max(vals, default=0), sum(vals))
    chosen_edges = ef.choose_edges_threshold(model, choose_threshold, len_choose)
    print("Chosen edges:", chosen_edges, len(chosen_edges[0]))
    layer.replace_many(*chosen_edges)

    if len(chosen_edges[0]) > 0:
        optim.add_param_group({'params': layer.embed_linears[-1].weight_values})
        # optim.add_param_group({'params': layer.weight_values})
    else:
        print("Empty metric")

    return {'max': max(vals, default=0), 'sum': sum(vals), 'len': len(vals), 'len_choose': layer.count_replaces[-1]}

In [7]:
# Define the model
class SimpleFCN(nn.Module):
    def __init__(self, input_size=28 * 28):
        super(SimpleFCN, self).__init__()
        self.fc1 = nn.Linear(input_size, 10)

    def forward(self, x):
        x = self.fc1(x)
        return x

In [8]:
# Dataset and Dataloader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

# Load dataset and split into train/validation sets
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [9]:
criterion = nn.CrossEntropyLoss()
metrics = [
    MagnitudeL2Metric(criterion),
    SNIPMetric(criterion),
    # GradientMeanEdgeMetric(criterion),
    PerturbationSensitivityEdgeMetric(criterion),
]
model = SimpleFCN()
sparse_model = convert_dense_to_sparse_network(model)
sparse_linear = deepcopy(sparse_model.fc1)
# sparse_model.fc1.weight_indices

In [10]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: vanyamironov to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [11]:
run = wandb.init(
    project="self-expanding-nets",
    name=f"replace=(auto epoch, threshold 0.15), lr=5e-5, magnetic l2 metric",
)


In [12]:
train_sparse_recursive(sparse_model, train_loader, val_loader, 64, metrics[0],
                       edge_replacement_func=edge_replacement_func_new_layer)

100%|██████████| 750/750 [00:02<00:00, 341.26it/s]


Epoch 1/64, Train Loss: 1.4805, Val Loss: 0.9977, Val Accuracy: 0.8192
{}


100%|██████████| 750/750 [00:02<00:00, 352.60it/s]


Epoch 2/64, Train Loss: 0.8088, Val Loss: 0.6845, Val Accuracy: 0.8537
{}


100%|██████████| 750/750 [00:02<00:00, 325.02it/s]


...

Epoch 22/64, Train Loss: 0.0284, Val Loss: 0.1090, Val Accuracy: 0.9693
{}


100%|██████████| 750/750 [00:11<00:00, 63.19it/s]


Epoch 23/64, Train Loss: 0.0245, Val Loss: 0.1056, Val Accuracy: 0.9701
{}


  1%|          | 9/750 [00:00<00:16, 45.86it/s]


KeyboardInterrupt: 

In [34]:
sparse_model.fc1.weight_values[sparse_model.fc1.weight_values > 1e-8].shape

torch.Size([2863])

In [24]:
sparse_model.fc1.weight_values[28*28*10 - 100:28*28*10]

tensor([ 6.2616e-09,  6.3876e-09,  1.3104e-09,  2.8574e-09, -2.1528e-01,
         2.8560e-09,  8.8170e-09,  4.7567e-09,  4.8345e-10,  1.7017e-09,
         9.7213e-09,  5.8228e-09,  8.6957e-09,  5.3322e-10, -2.1531e-01,
         9.8957e-09,  5.5075e-09,  3.7054e-09,  9.4094e-10,  4.1665e-09,
         5.9588e-09,  6.0718e-09,  9.6920e-09,  9.3723e-11, -1.7371e-01,
         1.5218e-09,  5.7825e-09,  7.9518e-09,  4.6619e-09,  4.2538e-09,
         5.6038e-09,  7.7219e-09,  2.7703e-09,  8.6223e-09, -1.8589e-01,
         7.8512e-09,  1.0788e-09,  3.0154e-09,  4.4037e-09,  1.0803e-09,
         8.7425e-09,  7.0480e-09,  4.9039e-09,  9.3589e-09,  2.0061e-01,
         6.3386e-09,  4.6793e-09,  4.4443e-09,  4.4504e-09,  6.0459e-09,
         5.1762e-09,  6.6578e-09,  9.6561e-09,  7.5907e-09, -1.8022e-01,
         4.7682e-09,  4.5682e-10,  8.1985e-09,  4.1293e-10,  5.0787e-09,
         1.5348e-09,  5.3798e-09,  5.0537e-09,  6.8203e-09, -2.2200e-01,
         5.9968e-09,  4.7665e-09,  2.5003e-09,  9.0

In [13]:
optimizer1 = optim.Adam(sparse_model.parameters(), lr=1e-4)